In [ ]:
# -*- coding: utf-8 -*-
"""SR2I208.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1kQcr0AWbVW19lgmQXtmCvO5-thzYjfyG
"""

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from math import sqrt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

"""# **Preprocessing des donnÃ©es:**"""

def read_data_SubClass(data) :
    
  #data = pd.read_csv(file_data)
  data = data.dropna()

  data["Data"].astype(str)
  data["Data"] = data["Data"].str.replace(" " , "")
  data["Data"] = data["Data"].apply(int , base = 16)

  data["Class"].replace(['Normal', 'Attack'],
                        [0, 1], inplace=True)
  
  uniques = data["SubClass"].unique()
  data["SubClass"].replace(uniques , [i for i in range(len(uniques))] , inplace = True)

  uniques_2 = data["SubClass"].unique()
  data["SubClass"].replace(uniques_2 , [i for i in range(len(uniques_2))] , inplace = True)

  data["Arbitration_ID"]  =  data["Arbitration_ID"].apply(int , base = 16)

  X = data[['Timestamp' ,'Arbitration_ID', 'DLC','Data']].values
  y = data["SubClass"].values
  
  return X , y

def read_data(data) :
    
  #data = pd.read_csv(file_data)
  data = data.dropna()

  data["Data"].astype(str)
  data["Data"] = data["Data"].str.replace(" " , "")
  data["Data"] = data["Data"].apply(int , base = 16)

    
  data["Class"].replace(['Normal', 'Attack'],
                        [0, 1], inplace=True)


  data["Arbitration_ID"]  =  data["Arbitration_ID"].apply(int , base = 16)

  X = data[['Timestamp' ,'Arbitration_ID', 'DLC','Data']].values
  y = data["Class"].values

  return X , y

# donnÃ©es de test :
list_test = ['Pre_submit_S.csv' , 'Pre_submit_D.csv', 'Fin_host_session_submit_S.csv']

df_test = pd.DataFrame()

for elt in list_test : 
    df = pd.read_csv(elt)
    frames = [df_test , df]
    df_test = pd.concat(frames)

list_train = ['Pre_train_D_1.csv' , 'Pre_train_S_1.csv' , 'Pre_train_S_2.csv', 'Pre_train_D_2.csv']

df_train = pd.DataFrame()

for elt in list_train : 
    df = pd.read_csv(elt)
    frames = [df_train , df]
    df_train = pd.concat(frames)

sc = StandardScaler()

"""#**Distribution de data dans les diffÃ©rents fichiers**"""

list_of_csv = ['Pre_train_D_0.csv' , 'Pre_train_D_1.csv' , 'Pre_train_D_2.csv' , 'Pre_train_S_0.csv' , 'Pre_train_S_1.csv' , 'Pre_train_S_2.csv']

print("Preliminary , training")
y_tot = 0
y_normal = 0
y_attack = 0
for elt in list_of_csv :
  data = pd.read_csv(elt)
  _ , y = read_data(data) 
  n = len(y)
  y_tot += n
  print("Number of data points in " , elt ," : " ,len(y))
  for i in np.unique(y) :
      print(i , list(y).count(i) )
      y_normal += (1-i)*list(y).count(i)
      y_attack += i*list(y).count(i)

  print("         --------**---------")

print("The total number of data points :" , y_tot)
print("Normal : " , y_normal)
print("Attack : " , y_attack)

print("Preliminary , submission")
y_tot = 0
y_normal = 0
y_attack = 0

for elt in ['Pre_submit_D.csv','Pre_submit_S.csv'] :
  data = pd.read_csv(elt)
  _ , y = read_data(data) 
  n = len(y)
  y_tot += n
  print("Number of data points in " , elt ," : " ,len(y))
  for i in np.unique(y) :
      print(i , list(y).count(i) )
      y_normal += (1-i)*list(y).count(i)
      y_attack += i*list(y).count(i)
  print("         --------**---------")

print("The total number of data points :" , y_tot)
print("Normal : " , y_normal)
print("Attack : " , y_attack)

data = pd.read_csv('Fin_host_session_submit_S.csv')

print("Final")
_ , y = read_data(data) 
n = len(y)

print("Number of data points in " , "Fin_host_session_submit_S.csv" ," : " ,len(y))
for i in np.unique(y) :
  print(i , list(y).count(i) )

"""# **2 classes:**"""

# des dictionnaires pour garder les scores et valeurs de f1 des diffÃ©rents algorithmes
dict_of_scores = {}
dict_of_f1 = {}

df_train.isnull().sum()

df_test.isnull().sum()

X_test , y_test = read_data(df_test)
X_train , y_train = read_data(df_train)

Counter(y_test)

Counter(y_train)

figure, axis = plt.subplots(1,2)
i=0
lab = 'train'

for elt in [y_train, y_test] :
   ratio = Counter(elt)
   labels = list(ratio.keys())
   sizes = list(ratio.values())
   
   axis[i].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
   axis[i].axis('equal')
   
   axis[i].set_title( 'Distribution in '+ lab + ' data' )
   lab= 'test'
   i+=1

plt.show()

undersample = RandomUnderSampler(sampling_strategy=1)
X_test, y_test = undersample.fit_resample(X_test, y_test)

X_train, y_train = undersample.fit_resample(X_train, y_train)

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

"""#**KMeans avec 2 classes**"""

X = sc.fit_transform(X_test)
y = y_test

Counter(y)

pca = PCA(n_components = 2)
X = pca.fit_transform(X)

model = KMeans(n_clusters = 2)

model.fit(X)

centers = model.cluster_centers_

plt.scatter(centers[:, 0], centers[:, 1], s=100, c='black')

labels = model.labels_

sns.scatterplot(x=X[:,0], y=X[:,1], c= labels, cmap='cool' )
sns.scatterplot(x=centers[:,0], y=centers[:,1],  cmap='black' )

sns.scatterplot(x=X[:,0], y=X[:,1],marker='o' ,c= y, cmap='cool' )

for s in np.unique(y) :
  print('Number of predicted in cluster' , s )
  print(s , np.sum(labels == s) )

for s in np.unique(y) :
  print('Real distribution of messages predicted as ' , s )
  #real labels of messages predicted as of label s
  real_of_predicted = y[labels == s]
  for i in np.unique(real_of_predicted) :
    print(i , np.sum(real_of_predicted == i) )

accuracy = accuracy_score(y, labels)
f1 = f1_score(y , labels)

print("Accuracy=", accuracy, "  f1=", f1)

dict_of_scores['k_means'] = accuracy
dict_of_f1['k_means'] = f1

"""# **K-NN et Decision Trees pour 2 classes**"""

Counter(y_train)

k_values= [3, 5, 7, 9]
for k in k_values :
  model = KNeighborsClassifier(n_neighbors= k)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  f1 = f1_score(y_test , y_pred , average='binary')

  print("Accuracy for k=", k , "is: ", accuracy_score(y_test, y_pred))
  print("Confusion matrix: " , confusion_matrix(y_test, y_pred))
  print("f1: " , f1)
  print("=================***====================")

"""**La meilleure valeur de k est k = 3.**"""

models = [ KNeighborsClassifier(n_neighbors=3), DecisionTreeClassifier()]

for model in models:
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test , y_pred , average='binary')

    dict_of_scores[str(model)] = score
    dict_of_f1[str(model)] = f1

    print("Accuracy of" , model , "is: ", score)
    print("F1 score of" , model , "is: ", f1)
    conf_matrix = confusion_matrix(y_test,y_pred)
    print("Confusion Matrix:\n",conf_matrix)
    print("\n==============***===============")

"""## **Courbe des scores et des valeurs de f1 pour les diffÃ©rents algorithmes**"""

models_ = list(dict_of_scores.keys())
accuracies = list(dict_of_scores.values())
f1s = list(dict_of_f1.values())

plt.plot(models_ , accuracies , color = 'r' , label='Accuracy')
plt.plot(models_ , f1s , color = 'black' , label='f1 score')
plt.title("Comparaison des résultats des differents modeles")
plt.legend()
plt.show()

"""# **Classification en 5 classes: normal et les 4 attaques**"""

Xm_test , ym_test = read_data_SubClass(df_test)
Xm_train , ym_train = read_data_SubClass(df_train)

Counter(ym_test)

Counter(ym_train)

figure, axis = plt.subplots(1,2)
i=0
lab = 'train'

for elt in [ym_train, ym_test] :
   ratio = Counter(elt)
   labels = list(ratio.keys())[1:]
   sizes = list(ratio.values())[1:]
   
   axis[i].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
   axis[i].axis('equal')
   
   axis[i].set_title( 'Attacks in '+ lab + ' data' )
   lab= 'test'
   i+=1

plt.show()

sc = StandardScaler()
Xm_train = sc.fit_transform(Xm_train)
Xm_test = sc.transform(Xm_test)

dict_scores = {}
dict_f1s = {}

"""#**KMeans avec 5 classes**"""

Xm = sc.fit_transform(Xm_test)
ym = ym_test

Counter(ym)

pca = PCA(n_components = 2)
Xm = pca.fit_transform(Xm)

model = KMeans(n_clusters = 5)

model.fit(Xm)

centers = model.cluster_centers_

plt.scatter(centers[:, 0], centers[:, 1], s=100, c='black')

labels = model.labels_

sns.scatterplot(x=Xm[:,0], y=Xm[:,1], c= labels, cmap='cool' )
sns.scatterplot(x=centers[:,0], y=centers[:,1],  cmap='black' )

sns.scatterplot(x=Xm[:,0], y=Xm[:,1],marker='o' ,c= ym, cmap='cool' )

for s in np.unique(ym) :
  print('Number of predicted in cluster' , s )
  print(s , np.sum(labels == s) )

for s in np.unique(ym) :
  print('Real distribution of messages predicted as ' , s )
  #real labels of messages predicted as of label s
  real_of_predicted = ym[labels == s]
  for i in np.unique(real_of_predicted) :
    print(i , np.sum(real_of_predicted == i) )

labels

# On va donner Ã  chaque cluster le label majoritaire : 
for i in range(len(labels)) : 
  
  if labels[i] == 1 :
    labels[i] = 3
  elif labels[i] == 2 :
    labels[i] = 4
  elif labels[i] == 3 :
    labels[i] = 2
  elif labels[i] == 4 :
    labels[i] = 1

labels

accuracy = accuracy_score(ym, labels)
f1 = f1_score(ym , labels,average=None)

print("Accuracy=", accuracy, "  f1=", f1)

dict_scores['K-means'] = accuracy
dict_f1s['Kmeans'] = f1

"""# **K-NN et Decision Trees pour 5 classes**"""

models = [KNeighborsClassifier(n_neighbors=3)  , DecisionTreeClassifier()]

for model in models:

    model.fit(Xm_train,ym_train)
    y_pred = model.predict(Xm_test)
    score = accuracy_score(ym_test, y_pred)
    f1 = f1_score(ym_test , y_pred , average=None)
    conf_matrix = confusion_matrix(ym_test,y_pred)

    dict_scores[str(model)] = score
    dict_f1s[str(model)] = f1

    print("Accuracy of" , model , "is: ", score)
    print("F1 score of" , model , "is: ", f1)
    print("Confusion Matrix:\n",conf_matrix)
    print("\n==============***===============")

models_ = list(dict_scores.keys())
accuracies = list(dict_scores.values())
f1s = list(dict_f1s.values())
colors_f1s = ['black' , 'b' , 'g' , 'y', 'm']
plt.plot(models_ , accuracies , color = 'r' , label='Accuracy')
for i in range(4) :
  plt.plot(models_ , [elt[i] for elt in f1s] , color = colors_f1s[i] , label=str('f1 score of '+str(i))

plt.title("Comparaison des résultats des différents modeles")
plt.legend()
plt.show()